In [ ]:
import branch_length
import distributions

import altair as alt
import numpy as np
import pandas as pd

alt.renderers.enable("notebook")

import importlib 
importlib.reload(branch_length)
importlib.reload(distributions)

In [ ]:
def plot_functions(f_true, f_approx):
    x_vals = np.linspace(0.1, 5, 40)
    x_transpose = np.transpose(np.array([x_vals]))
    data = pd.DataFrame({"x": x_vals, "truth": f_true(np.transpose(np.array([x_vals]))), 
                         "approx": f_approx(np.transpose(np.array([x_vals])))})
    return alt.Chart(data.melt(id_vars=["x"])).mark_line().encode(
        x='x',
        y='value',
        color='variable'
    )

In [ ]:
stepsz = 0.05
grad_clip = 100.
d = distributions.Normal(1)
true_loc = np.array([2.])
true_shape = np.array([1.])
phylo_log_like = lambda x: d.log_prob(x, true_loc, true_shape)
phylo_log_like_grad = lambda x: d.log_prob_grad(x, true_loc, true_shape)

loc = np.array([1.9])
shape = np.array([0.95])

In [ ]:
x = d.sample(loc, shape, 1)
x_transpose = np.transpose(np.array([x]))
loc_grad, shape_grad = branch_length.param_grad(d, phylo_log_like_grad(x), x, loc, shape)

print("before")
print(true_loc, loc, loc_grad*stepsz)
print(true_shape, shape, shape_grad*stepsz)

loc += np.clip(loc_grad*stepsz, -grad_clip, grad_clip)
shape += np.clip(shape_grad*stepsz, -grad_clip, grad_clip)

print("after")
print(true_loc, loc, loc_grad*stepsz)
print(true_shape, shape, shape_grad*stepsz)

plot_functions(phylo_log_like, lambda x: d.log_prob(x, loc, shape))